# KNN classification

This notebook aims to solve task 4 by using a KNN classifier.

In [16]:
import pandas as pd
from os import path
import numpy as np
from preprocessing import get_train_test_data

X_train, y_train, X_test, y_test, columns_to_keep = get_train_test_data()

In [9]:
X_train = X_train.dropna()
X_test = X_test.dropna()
y_train = y_train[X_train.index]
y_test = y_test[X_test.index]

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

Grid of hyperparameters
param_grid = {
    'n_neighbors': [3, 5, 7, 9],        # number of neighbors
    'weights': ['uniform', 'distance'],  # uniformal weights or based on the distance
    'algorithm': ['auto'],  # algorithm chosen by the model
    'metric': ['euclidean'],  # metrics
}

# knn model
knn = KNeighborsClassifier()

# Grid search
grid_search = GridSearchCV(
    estimator=knn,
    param_grid=param_grid,
    scoring='f1',  # Use f1-score
    cv=4,          # 4-fold cross-validation
    verbose=1,     # Print the informations
    n_jobs=-1      # Use all the cores
)

# Execute the grid search
grid_search.fit(X_train, y_train)

# Best Model
best_model = grid_search.best_estimator_

# Print the best parameters
print("Best parameters:", grid_search.best_params_)

# Print the best f1-score
print("Best f1 score:", grid_search.best_score_)


Fitting 4 folds for each of 8 candidates, totalling 32 fits


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best parameters: {'algorithm': 'auto', 'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'distance'}
Best f1 score: 0.48291895475759183


In [13]:
test_pred_knn = best_model.predict(X_test)

In [15]:
from sklearn.metrics import classification_report
print(classification_report(y_test,
                            test_pred_knn,
                            target_names=['0', '1']))

              precision    recall  f1-score   support

           0       0.87      0.90      0.89     30219
           1       0.29      0.24      0.26      5187

    accuracy                           0.80     35406
   macro avg       0.58      0.57      0.57     35406
weighted avg       0.79      0.80      0.79     35406

